# L'approssimazione di una funzione lineare mediante un singolo neurone a comportamento lineare

Luca Mari, gennaio 2025  

Quest'opera è distribuita con <a href="http://creativecommons.org/licenses/by-nc-sa/4.0" target="_blank">Licenza Creative Commons Attribuzione - Non commerciale - Condividi allo stesso modo 4.0 Internazionale</a>.  
<img src="https://creativecommons.it/chapterIT/wp-content/uploads/2021/01/by-nc-sa.eu_.png" width="100">

**Obiettivo**: comprendere, a partire da un esempio concreto, che una rete neurale deve includere degli elementi non lineari per poter approssimare appropriatamente anche delle semplici funzioni non lineari.  
**Precompetenze**: basi di Python; almeno qualche idea di analisi matematica.

> Per eseguire questo notebook, supponiamo con VSCode, occorre:
> * installare un interprete Python
> * scaricare da https://code.visualstudio.com/download e installare VSCode
> * eseguire VSCode e attivare le estensioni per Python e Jupyter
> * ancora in VSCode:
>     * creare una cartella di lavoro e renderla la cartella corrente
>     * copiare nella cartella il file di questa attività: [neuron.ipynb](neuron.ipynb)
>     * aprire il notebook `neuron.ipynb`
>     * creare un ambiente virtuale locale Python (Select Kernel | Python Environments | Create Python Environment | Venv, e scegliere un interprete Python):
>     * installare il modulo Python richiesto, eseguendo dal terminale:  
>         `pip install torch`

Una rete neurale è l'implementazione di una funzione parametrica $Y = f(X; K)$, e può essere intesa come uno strumento di approssimazione di funzioni $F(X)$ date: attraverso un opportuno addestramento, si trovano i valori appropriati dei parametri $K$ in modo che $f(X; K) \approx F(X)$.

Quest'idea venne sviluppata inizialmente assumendo che i componenti elementari di una rete -- i suoi neuroni -- avessero un comportamento lineare:  

![rete](neuron.drawio.svg)  
nel caso di due input.

La situazione più semplice è ovviamente quella di una rete costituita da un solo neurone. Facciamo qualche prova.

Per costruire e operare sulla rete useremo `PyTorch`: importiamo perciò i moduli Python che saranno necessari.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

Costruiamo la rete usando `PyTorch` (il codice ha un po' di dettagli tecnici, non necessariamente importanti: i commenti potrebbero essere comunque utili) e visualizziamo i valori dei suoi parametri, che inizialmente sono casuali.

In [2]:
class OneNeuron(nn.Module):
    def __init__(self):
        super(OneNeuron, self).__init__()
        self.neuron = nn.Linear(2, 1)

        self.loss = nn.MSELoss()        # funzione di errore: Mean Squared Error
        self.optimizer = optim.SGD(self.parameters(), lr=0.01) # ottimizzatore: Stochastic Gradient Descent # type: ignore

    def forward(self, x):
        x = self.neuron(x)
        return x

    def set_learning_rate(self, learning_rate):
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = learning_rate        

    def train(self, x, y, epochs, repeat):
        print(f'\n*** Addestramento ***\nepoca\terrore)')
        for epoch in range(epochs):
            self.optimizer.zero_grad()  # azzera i gradienti
            output = self(x)            # calcola l'output
            loss = self.loss(output, y) # calcola la funzione di errore
            loss.backward()             # calcola i gradienti
            self.optimizer.step()       # aggiorna i valori dei parametri
            if (epoch+1) % repeat == 0:
                print(f'{epoch+1}\t{loss.item():.3f}')

    def predict(self, examples, fun):
        print('\n*** Inferenza ***')
        x_test = examples
        y_test = self(x_test)           # calcola la previsione
        y_true = self.calc_fun(fun, x_test)
        print('x1\tx2\ty\ty prev\terrore')
        for i in range(x_test.size(0)):
            x1, x2 = x_test[i][0].item(), x_test[i][1].item()
            y, y_hat = y_true[i].item(), y_test[i].item()
            print(f'{x1:.2f}\t{x2:.2f}\t{y:.2f}\t{y_hat:.2f}\t{y - y_hat:.2f}')
        print(f'Errore quadratico medio: {torch.mean((y_test - y_true)**2):.5f}')

    def reset_parameters(self):
        for layer in self.children():
            if hasattr(layer, 'reset_parameters'):
                layer.reset_parameters()

    def print_parameters(self):
        for name, param in self.named_parameters():
            print(name, param.data)

    def calc_fun(self, fun, X):
        return fun(X[:, 0], X[:, 1]).view(-1, 1)


model = OneNeuron()
print('I parametri della rete sono:'); model.print_parameters()

I parametri della rete sono:
neuron.weight tensor([[ 0.4909, -0.6757]])
neuron.bias tensor([-0.2738])


Costruiamo il training set, prima di tutto negli input (_features_, _covariates_) come un certo numero di coppie di numeri casuali.

In [3]:
def examples(n): return (10 * torch.rand(n, 2) - 5) # genera n esempi nella forma ognuno di una coppia di numeri casuali tra -5 e 5
num_examples = 100                      # numero di esempi per il training set
X = examples(num_examples)              # calcola i valori degli esempi: input del training set

Scegliamo la funzione, dunque a due argomenti, da approssimare. Essendo un caso di _supervised learning_, calcoliamo la funzione per tutte le coppie del training set e aggiungiamo il risultato al training set stesso.

In [5]:
def fun(x1, x2): return (x1 + x2) / 2   # funzione da approssimare, in questo caso la media tra due numeri
Y = model.calc_fun(fun, X)              # calcola il valore della funzione per ogni esempio: output del training set
i = random.randint(0, num_examples-1)   # indice casuale
print(f"L'esempio di una tripla nel training set: x1={X[i, 0].item():.2f}, x2={X[i, 1].item():.2f}, y={Y[i, 0].item():.2f}")

L'esempio di una tripla nel training set: x1=1.02, x2=-4.57, y=-1.77


Già ora possiamo mettere in funzione la rete, su un certo numero di esempi che costituiscono dunque un test set, ma ovviamente il risultato non sarà in alcun modo accurato.

In [6]:
model.predict(examples(10), fun)        # inferenza prima dell'addestramento


*** Inferenza ***
x1	x2	y	y prev	errore
3.83	-3.28	0.27	3.82	-3.55
2.72	-1.99	0.36	2.41	-2.05
-0.49	2.20	0.86	-2.00	2.86
3.98	-1.00	1.49	2.35	-0.86
-0.17	4.99	2.41	-3.73	6.14
-1.56	3.85	1.14	-3.64	4.78
1.05	-3.57	-1.26	2.65	-3.91
-3.77	-0.79	-2.28	-1.59	-0.69
-2.65	2.44	-0.11	-3.23	3.12
3.42	-2.48	0.47	3.08	-2.61
Errore quadratico medio: 11.86144


Addestriamo allora la rete, dopo aver assegnato valori opportuni ai due iperparametri fondamentali:  
-- il numero di volte in cui il processo di addestramento viene ripetuto, e  
-- la velocità di apprendimento (_learning rate_).

In [7]:
num_epochs = 100                        # numero di ripetizioni del processo di addestramento
repeat = 10                             # numero di ripetizioni dopo le quali visualizzare l'errore
model.reset_parameters()                # reinizializza i parametri della rete
model.set_learning_rate(0.02)           # imposta il learning rate
model.train(X, Y, num_epochs, repeat)   # addestra la rete


*** Addestramento ***
epoca	errore)
10	0.141
20	0.062
30	0.028
40	0.012
50	0.006
60	0.002
70	0.001
80	0.000
90	0.000
100	0.000


/home/lucamari/Bin/tutorial/.venv/lib/python3.13/site-packages/torch/autograd/graph.py:823: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Mettiamo in funzione la rete su un nuovo test set: se l'addestramento ha avuto successo, si dovrebbe ottenere un piccolo errore quadratico medio.

In [8]:
model.predict(examples(10), fun)        # inferenza dopo l'addestramento


*** Inferenza ***
x1	x2	y	y prev	errore
-1.65	0.11	-0.77	-0.76	-0.01
3.80	3.68	3.74	3.75	-0.01
-2.48	-1.38	-1.93	-1.92	-0.01
-2.31	-1.88	-2.10	-2.09	-0.01
-3.64	4.79	0.58	0.59	-0.01
4.17	-0.34	1.91	1.92	-0.01
1.62	-2.67	-0.52	-0.51	-0.01
1.06	-4.45	-1.69	-1.68	-0.01
-2.98	0.58	-1.20	-1.19	-0.01
2.10	-2.12	-0.01	0.00	-0.01
Errore quadratico medio: 0.00010


Visualizziamo i valori dei parametri della rete: se l'addestramento ha avuto successo, dovrebbero essere vicini ai valori attesi.

In [9]:
model.print_parameters()

neuron.weight tensor([[0.5004, 0.4998]])
neuron.bias tensor([0.0096])


La struttura della rete è così semplice che possiamo ripetere l'intero processo senza ricorrere a `PyTorch`, per mostrare così in modo esplicito la logica della procedura.

In [10]:
num_epochs = 100                        # numero di ripetizioni del processo di addestramento
repeat = 10                             # numero di ripetizioni dopo le quali visualizzare l'errore
learning_rate = 0.01                    # learning rate
minibatch_size = 10                     # dimensione del minibatch: numero di esempi estratti dal training set per ogni epoca

k0, k1, k2 = torch.randn(3)             # valori casuali di inizializzazione dei parametri 

print(f'\n*** Addestramento ***\nepoca\terrore\tk0\tk1\tk2')
for i in range(num_epochs):
    indexes = torch.randperm(X.size(0))[:minibatch_size]            # seleziona in modo casuale gli indici del minibatch
    X1 = X[indexes, 0]                                              # estrai dal training set gli argomenti della funzione
    X2 = X[indexes, 1]
    Y_prev = k0 + k1 * X1 + k2 * X2                                 # calcola la previsione
    Y_true = Y[indexes, 0]                                          # estrai dal training set il valore della funzione
    loss = torch.mean((Y[indexes, 0] - Y_prev)**2)                  # calcola la funzione di errore (errore quadratico medio)
    k0 -= learning_rate * 2 * torch.mean(Y_prev - Y_true)           # calcola le derivate parziali della funzione di errore...
    k1 -= learning_rate * 2 * torch.mean((Y_prev - Y_true) * X1)    # ... e aggiorna i valori dei parametri...
    k2 -= learning_rate * 2 * torch.mean((Y_prev - Y_true) * X2)    # ... dunque "scendendo lungo il gradiente"
    if (i+1) % repeat == 0:
        print(f'{i+1}\t{loss.item():.3f}\t{k0:.3f}\t{k1:.3f}\t{k2:.3f}')


*** Addestramento ***
epoca	errore	k0	k1	k2
10	3.272	-0.897	0.202	0.541
20	0.396	-0.747	0.408	0.589
30	0.371	-0.615	0.475	0.526
40	0.249	-0.503	0.480	0.522
50	0.163	-0.411	0.481	0.516
60	0.117	-0.335	0.499	0.497
70	0.078	-0.274	0.498	0.503
80	0.040	-0.225	0.480	0.510
90	0.029	-0.186	0.484	0.505
100	0.028	-0.152	0.498	0.505


Quella che segue è invece un'implementazione semplificata di un algoritmo genetico, per risolvere lo stesso problema di ottimizzazione.

In [11]:
num_individuals = 100                   # numero di individui della popolazione in evoluzione
num_survivors = 50                      # numero di individui che in ogni epoca sopravvive
num_mutations = 5                       # numero di individui che in ogni epoca subisce una mutazione 
width_mutations = .1                    # ampiezza (deviazione standard) delle mutazioni
minibatch_size = 10                     # dimensione del minibatch: numero di esempi estratti dal training set per ogni epoca
num_epochs = 100                        # numero di ripetizioni del processo di addestramento
repeat = 10                             # numero di ripetizioni dopo le quali visualizzare l'errore

k = torch.randn(num_individuals, 3)

print(f'\n*** Addestramento ***\nepoca\terrore\tk0\tk1\tk2')
for i in range(num_epochs):
    indexes = torch.randperm(X.size(0))[:minibatch_size]            # seleziona in modo casuale gli indici del minibatch
    X1 = X[indexes, 0].view(-1, 1).T                                # estrai dal training set gli argomenti della funzione
    X2 = X[indexes, 1].view(-1, 1).T
    Y_true = Y[indexes, 0].view(-1, 1).T                            # estrai dal training set il valore della funzione
    Y_prev = k[:,0].view(-1, 1) + k[:,1].view(-1, 1) * X1 + k[:,2].view(-1, 1) * X2 # calcola la previsione

    loss = torch.mean((Y[indexes, 0] - Y_prev)**2, dim=1)           # calcola la funzione di errore per ogni individuo
    sorted_indexes = torch.argsort(loss, descending=False)          # ottieni gli indici degli individui ordinati in base all'errore
    k = k[sorted_indexes][:num_survivors]                           # ordina gli individui per fitness e seleziona i migliori

    m0 = torch.randint(num_survivors, (num_mutations, 1)).view(-1)  # seleziona casualmente gli indici degli individui da mutare
    m1 = torch.randint(3, (num_mutations, 1)).view(-1)
    k[m0, m1] += torch.randn(num_mutations) * width_mutations       # introduci una mutazione negli individui selezionati

    k = torch.cat((k, torch.randn(num_individuals - num_survivors, 3)), 0) # reintegra la popolazione con nuovi individui casuali

    if (i+1) % repeat == 0:
        best = k[sorted_indexes][0]
        print(f'{i+1}\t{loss[sorted_indexes][0].item():.3f}\t{best[0].item():.3f}\t{best[1].item():.3f}\t{best[2].item():.3f}')




*** Addestramento ***
epoca	errore	k0	k1	k2
10	0.043	-0.200	0.453	0.568
20	0.146	-0.200	0.395	0.446
30	0.081	-0.257	0.466	0.668
40	0.134	-0.289	0.540	0.525
50	0.009	-0.050	0.534	0.501
60	0.018	-0.097	0.598	0.501
70	0.100	-0.257	0.497	0.428
80	0.057	-0.033	0.662	0.459
90	0.057	-0.347	0.406	0.411
100	0.085	0.922	0.519	0.431


Tornando ora a usare `PyTorch`: d'altra parte, è evidente che un singolo neurone a comportamento lineare può approssimare efficacemente solo funzioni molto semplici. Anche aumentando il numero di esempi e di ripetizioni del processo di addestramento, per esempio non è in grado di approssimare in modo accettabile la funzione massimo tra due numeri.

In [12]:
num_examples = 1000                     # numero di esempi per il training set
X = examples(num_examples)              # input del training set
def fun(x1, x2): return torch.max(x1, x2) # funzione da approssimare, in questo caso il massimo tra due numeri
Y = model.calc_fun(fun, X)              # calcola il valore della funzione per ogni esempio: output del training set
num_epochs = 1000                       # numero di ripetizioni del processo di addestramento
repeat = 100                            # numero di ripetizioni dopo le quali visualizzare l'errore
model.reset_parameters()                # reinizializza i parametri della rete
model.set_learning_rate(0.01)           # imposta il learning rate
model.train(X, Y, num_epochs, repeat)   # addestra la rete
model.predict(examples(10), fun)        # metti in funzione la rete


*** Addestramento ***
epoca	errore)
100	1.491
200	1.413
300	1.411
400	1.411
500	1.411
600	1.411
700	1.411
800	1.411
900	1.411
1000	1.411

*** Inferenza ***
x1	x2	y	y prev	errore
0.66	0.07	0.66	2.06	-1.40
-4.36	-0.82	-0.82	-0.91	0.09
-1.24	-2.52	-1.24	-0.21	-1.03
-4.02	-2.73	-2.73	-1.71	-1.02
-1.26	-1.79	-1.26	0.15	-1.41
0.26	3.94	3.94	3.83	0.11
0.90	2.30	2.30	3.31	-1.01
-1.82	-2.16	-1.82	-0.32	-1.50
1.40	1.20	1.40	3.01	-1.61
4.66	-3.45	4.66	2.27	2.39
Errore quadratico medio: 1.76398


Per ottenere approssimazioni accettabili occorre dunque costruire una rete più complessa.